In [1]:
import argparse
import logging

import adapters
import pandas as pd
import torch
from transformers import AutoTokenizer

from dataset_lib import SumDataLoader
from utils import generate_summary, get_pretrained_model, MODEL_ID, rouge_metric, LLaMAModelClass, \
    convert_params_to_bfloat16



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/

/Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
lora_path = "saved_models/medical_lora_hf/medical_lora"
ia3_path = "saved_models/medical_ia3_hf/medical_ia3"

In [3]:
domain = "medical"

In [4]:
llama = LLaMAModelClass(version=3.0, instruct_mode=False, quantization_config=None, model_checkpoint=None, mlm=False)
print(llama.model)

Loading LLaMA's: TinyLlama/TinyLlama-1.1B-Chat-v1.0 Model for the process.


/Users/mehul/TUM/SS24/DomainAdaptationLLMs/311_venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


*** Model Loaded ***
Loading LLaMA's: TinyLlama/TinyLlama-1.1B-Chat-v1.0 Tokenizer for the process.
*** Tokenizer Loaded ***
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Lla

In [5]:
from peft import PeftModel
# llama.model = PeftModel.from_pretrained(llama.model, lora_path, adapter_name="medical_lora")
# print(llama.model)
llama.model.load_adapter(lora_path, adapter_name="m_lora")
print(llama.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (m_lora): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (m_lora): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (m_lora): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256, bia

In [7]:
llama.model.load_adapter(ia3_path, adapter_name="m_ia3", offload_folder="offload_dir")
print(llama.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(
            (base_layer): lora.Linear(
              (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
              (lora_dropout): ModuleDict(
                (m_lora): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (m_lora): Linear(in_features=2048, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (m_lora): Linear(in_features=8, out_features=2048, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict()
            )
            (ia3_l): ParameterDict(  (m_ia3): Parameter containing: [torch.mps.FloatTensor of size 2048x1])
          

In [31]:
llama.model.set_adapter(["m_lora", "m_ia3"])

In [32]:
print(llama.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(
            (base_layer): lora.Linear(
              (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
              (lora_dropout): ModuleDict(
                (m_lora): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (m_lora): Linear(in_features=2048, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (m_lora): Linear(in_features=8, out_features=2048, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict()
            )
            (ia3_l): ParameterDict(  (m_ia3): Parameter containing: [torch.mps.FloatTensor of size 2048x1])
          

In [29]:
llama.model.disable_adapters()

In [33]:
llama.model.active_adapters()

['m_lora', 'm_ia3']

In [34]:
print(llama.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(
            (base_layer): lora.Linear(
              (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
              (lora_dropout): ModuleDict(
                (m_lora): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (m_lora): Linear(in_features=2048, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (m_lora): Linear(in_features=8, out_features=2048, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict()
            )
            (ia3_l): ParameterDict(  (m_ia3): Parameter containing: [torch.mps.FloatTensor of size 2048x1])
          

In [15]:
content = """ Rome had begun expanding shortly after the founding of the Republic in the 6th century BC, though it did not expand outside the Italian Peninsula until the 3rd century BC, during the Punic Wars, afterwhich the Republic expanded across the Mediterranean.[5][6][7][8] Civil war engulfed Rome in the mid-1st century BC, first between Julius Caesar and Pompey, and finally between Octavian (Caesar's grand-nephew) and Mark Antony. Antony was defeated at the Battle of Actium in 31 BC, leading to the annexation of Egypt. In 27 BC, the Senate gave Octavian the titles of Augustus ("venerated") and Princeps ("foremost"), thus beginning the Principate, the first epoch of Roman imperial history. Augustus' name was inherited by his successors, as well as his title of Imperator ("commander"), from which the term "emperor" is derived. Early emperors avoided any association with the ancient kings of Rome, instead presenting themselves as leaders of the Republic.\nThe success of Augustus in establishing principles of dynastic succession was limited by his outliving a number of talented potential heirs; the Julio-Claudian dynasty lasted for four more emperors—Tiberius, Caligula, Claudius, and Nero—before it yielded in AD 69 to the strife-torn Year of the Four Emperors, from which Vespasian emerged as victor. Vespasian became the founder of the brief Flavian dynasty, to be followed by the Nerva–Antonine dynasty which produced the "Five Good Emperors": Nerva, Trajan, Hadrian, Antoninus Pius and the philosophically inclined Marcus Aurelius. In the view of the Greek historian Cassius Dio, a contemporary observer, the accession of the emperor Commodus in AD 180 marked the descent "from a kingdom of gold to one of rust and iron"[9]—a famous comment which has led some historians, notably Edward Gibbon, to take Commodus' reign as the beginning of the decline of the Roman Empire. """

In [17]:
from dataset_lib import llama3_testing_prompt, chat_template_prompt_inference
terminators = [
        llama.tokenizer.eos_token_id,
        llama.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
content = llama3_testing_prompt(content)

In [18]:
content

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nSummarize the given article by addressing the following key points:\n\n    1. Main Topic: What is the primary subject or theme of the article?\n\n    2. Context: What background information is provided to frame the discussion?\n\n    3. Key Arguments: What are the main arguments or points made by the author?\n\n    4. Evidence and Examples: What supporting evidence or examples are presented?\n\n    5. Conclusions: What conclusions does the author draw from the discussion?\n\n    6. Implications: What are the potential implications or significance of the article\'s content for its audience?<|eot_id|>\n\n            <|start_header_id|>user<|end_header_id|>\nPlease provide the summary for the article:\n Rome had begun expanding shortly after the founding of the Republic in the 6th century BC, though it did not expand outside the Italian Peninsula until the 3rd century BC, during the Punic Wars, afterwhich the Republic expanded 

In [20]:
device = "mps" if torch.backends.mps.is_available else ("cuda" if torch.cuda.is_available() else "cpu")
inputs = llama.tokenizer(content, return_tensors="pt").to(device)

In [35]:
llama.model.active_adapters()

['m_lora', 'm_ia3']

In [22]:
in_len = len(inputs["input_ids"][0])

In [40]:
with torch.inference_mode():
    summary_ids = llama.model.generate(**inputs,
                                 # max_length=512, 
                                 do_sample=True,  # Enable sampling
                                 top_k=50,  # Top-k sampling
                                 num_return_sequences=1,  # Generate a single sequence
                                 eos_token_id=terminators,
                                 # early_stopping=True,
                                 temperature=0.6,
                                 max_new_tokens=256)

In [43]:
summary_both = llama.tokenizer.decode(summary_ids[0][in_len:], skip_special_tokens=True)

In [44]:
print(summary_ia3)
print("\n->",summary_both)


Rome had begun expanding shortly after the founding of the Republic in the 6th century BC, and expanded across the Mediterranean during the Punic Wars. Civil war engulfed Rome in the mid-1st century BC, leading to the annexation of Egypt. In 27 BC, the Senate gave Augustus the titles of Augustus and Princeps, and he began the Principate era. Augustus' successors established dynastic succession principles, but outlived a number of talented potential heirs. The Julio-Claudian dynasty lasted for four emperors, including Tiberius, Caligula, Claudius, and Nero. The Flavian dynasty was founded by Vespasian, followed by the Nerva–Antonine dynasty, which produced the "Five Good Emperors": Nerva, Trajan, Hadrian, Antoninus Pius, and Marcus Aurelius. The accession of Commodus marked the beginning of the decline of the Roman Empire.

-> Rome had a long history of expansion and territorial conquests, beginning with the founding of the Republic in the 6th century BC and continuing through the Puni